In [1]:
!pip install dlib

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install mediapipe

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import cv2
import dlib
import numpy as np

# DLIB

In [2]:

# Load the face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # Download from dlib

In [30]:
# Load the face image and the mask
face_img = cv2.imread("face.jpg")
# face_img = cv2.imread("/projectnb/cs585bp/students/dlgirija/colorferet_extracted/00002/00002_930831_fa.ppm")
mask_img = cv2.imread("Surgical-Mask.png", cv2.IMREAD_UNCHANGED)  # Ensure mask has an alpha channel

# Convert to grayscale and detect faces
gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
faces = detector(gray)

for face in faces:
    landmarks = predictor(gray, face)
    
    # Get coordinates for mask placement (adjust as needed)
    top_nose = (landmarks.part(19).x, landmarks.part(19).y)
    chin = (landmarks.part(57).x, landmarks.part(57).y)
    left_cheek = (landmarks.part(0).x, landmarks.part(0).y)
    right_cheek = (landmarks.part(16).x, landmarks.part(16).y)

    # Calculate mask size
    mask_width = int(np.linalg.norm(np.array(left_cheek) - np.array(right_cheek)))
    mask_height = int(np.linalg.norm(np.array(top_nose) - np.array(chin))) * 2  # Scale factor

    # Resize the mask
    mask_resized = cv2.resize(mask_img, (mask_width, mask_height))

    # Get region of interest (ROI) in face image
    x1, y1 = left_cheek[0], top_nose[1]
    x2, y2 = x1 + mask_width, y1 + mask_height
    roi = face_img[y1:y2, x1:x2]

    # Extract mask alpha channel for blending
    mask_alpha = mask_resized[:, :, 3] / 255.0
    mask_rgb = mask_resized[:, :, :3]
    mask_rgb_resized = cv2.resize(mask_rgb, (roi.shape[1], roi.shape[0]))
    mask_alpha_resized = cv2.resize(mask_alpha, (roi.shape[1], roi.shape[0]))  # (Width, Height)
    

    # Blend the mask with the face
    for c in range(3):
        # print(1-mask_alpha)
        roi[:, :, c] = roi[:, :, c] * (1 - mask_alpha_resized) + mask_rgb_resized[:, :, c] * mask_alpha_resized

    # Place the modified ROI back into the image
    face_img[y1:y2, x1:x2] = roi

# Show the output
# cv2.imshow("Masked Face", face_img)
cv2.imwrite("output.jpg", face_img)  # Saves 'image' as "output.jpg"
# cv2.waitKey(0)
# cv2.destroyAllWindows()


True

In [46]:
import bz2
import cv2
import numpy as np

# Decompress .ppm.bz2 file
with bz2.BZ2File("/projectnb/cs585bp/students/dlgirija/colorferet/images/00001/00001_930831_fb_a.ppm.bz2", "rb") as f:
    decompressed_data = f.read()

# Convert the decompressed data into a numpy array
ppm_array = np.frombuffer(decompressed_data, dtype=np.uint8)

# Decode the PPM image using OpenCV
img = cv2.imdecode(ppm_array, cv2.IMREAD_COLOR)
cv2.imwrite("detectos.jpg", img)

True

In [49]:
from PIL import Image
import numpy as np

# Open image using PIL
# img = Image.open("image.ppm")

# Convert to OpenCV format (NumPy array)
# img_cv2 = np.array(img)
face_img = cv2.imread("face.jpg")
# face_img = cv2.imread("/projectnb/cs585bp/students/dlgirija/colorferet_extracted/00001/00001_930831_fa_a.ppm")
# img = Image.open("/projectnb/cs585bp/students/dlgirija/colorferet_extracted/00001/00001_930831_fa_a.ppm")
# face_img = np.array(img)
for i in range(68):  # 68 landmarks for dlib
    x, y = landmarks.part(i).x, landmarks.part(i).y
    cv2.circle(face_img, (x, y), 2, (0, 0, 255), -1)
cv2.imwrite("detectos.jpg", face_img)

True

# MediaPipe

## Detection

In [3]:
import cv2
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# cap = cv2.imread('face.jpg')
cap = cv2.imread("/projectnb/cs585bp/students/dlgirija/colorferet_extracted/00001/00001_930831_hr_a.ppm")
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    frame = cap
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            # cv2.putText(frame, str(idx), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)
            mp_drawing.draw_landmarks(frame, landmarks, mp_face_mesh.FACEMESH_CONTOURS)

    cv2.imwrite('detectos.jpg', frame)


W0000 00:00:1740165668.129923   90905 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740165668.139516   90907 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [70]:
import cv2
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Load the image
image_path = "face.jpg"  # Replace with your image path
image = cv2.imread(image_path)
h, w, _ = image.shape  # Get image dimensions

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

results = face_mesh.process(image_rgb)

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        for idx, landmark in enumerate(face_landmarks.landmark):
            x, y = int(landmark.x * w), int(landmark.y * h)  # Convert to image coordinates
            
            cv2.putText(image, str(idx), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
            cv2.circle(image, (x, y), 1, (255, 0, 0), -1)  # Draw small dot on each landmark

# Display the result
cv2.imwrite('Detectors.jpg', image)


W0000 00:00:1740162945.945170 1391524 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740162945.964192 1391527 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


True

## Masking

In [43]:
### import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Load image
# cap = cv2.imread("/projectnb/cs585bp/students/dlgirija/colorferet_extracted/00001/00001_930831_fa_a.ppm")
cap = cv2.imread('face.jpg')
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    frame = cap
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            # Define the landmark indices for the mouth and nose region
            MOUTH_INDICES = [
                93, 137, 123, 50, 101, 100, 47, 114, 122, 188, 6, # left
                351, 412, 343, 277, 329, 330, 280, 352, 366,323, # right
            ]
            NOSE_INDICES = []
            # Get landmark coordinates for mask area
            h, w, _ = frame.shape
            mask_points = []
            frame_points = []
            for idx in MOUTH_INDICES + NOSE_INDICES:
                x = int(landmarks.landmark[idx].x * w)
                y = int(landmarks.landmark[idx].y * h)
                mask_points.append((x, y))
                frame_points.append(idx)

            left_ind = [132,215,172,136,150,149,176,148,152] # left
            right_ind = [377,400,378,379,365,397,288,361] # right

            for idx in left_ind:
                x = int(landmarks.landmark[idx].x * w) - 15
                y = int(landmarks.landmark[idx].y * h) + 10
                mask_points.append((x, y))
                frame_points.append(idx)

            for idx in right_ind:
                x = int(landmarks.landmark[idx].x * w) + 15
                y = int(landmarks.landmark[idx].y * h) + 10
                mask_points.append((x, y))
                frame_points.append(idx)

            mask = frame
            hull = cv2.convexHull(np.array(mask_points, np.int32))
            cv2.fillConvexPoly(mask, hull, (100, 100, 100))
            frame = mask
            
    cv2.imwrite("Masked_Image.jpg", frame)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


W0000 00:00:1740168518.202766   96009 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740168518.220213   96018 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [19]:
 # 132,#, 377
                # 377,
                
                # 51, 174, 217, 126, 100, 101, 50, 123, 132,
                # 114,3, 47, 100, 101, 234, 50, 123,
                # 281, 363, 420, 355, 329, 330, 280, 352,#347, 346, 345,
                # 197, 419, 399, 343, 350, 451, 450, 448, 449, 261, 265
                # 401, 433, 435, 288, 367, 379, 401, 323, 366, 376, 454, 425, 426, 436, 427, 
                # 61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 78, 191, 80, 81, 82, 
                # 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 377, 152, 400, 148, 213, 214, 192, 138, 32, 74, 397, 365

# MOUTH_INDICES += [i for i in range()]
            # NOSE_INDICES = [2, 98, 327, 168]  # Nose bridge and tip
            # NOSE_INDICES = [197]#, 5, 4, 1] 

# cv2.fillPoly(mask, [np.array(mask_points, np.int32)], (100, 100, 100))  # Black-out mask area
            # for i in range(len(mask_points)):
            #     cv2.putText(mask, str(frame_points[i]), mask_points[i], cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)
            #     cv2.circle(mask, mask_points[i], 2, (0, 0, 255), -1)
            # Apply mask

# Create mask and fill region
            # mask = np.ones_like(frame) * 255  # White mask

In [24]:
mask.shape

(980, 1372, 3)

In [22]:
texture =cv2.imread('grass_texture.jpeg')

In [27]:
texture[0: mask.shape[0], 0:mask.shape[1], :].shape

(980, 1372, 3)

## Texture

In [49]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Load image
cap = cv2.imread('face.jpg')
texture = cv2.imread('grass_texture.jpeg')

with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    frame = cap
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            # Define the landmark indices for the mouth and nose region
            MOUTH_INDICES = [
                93, 137, 123, 50, 101, 100, 47, 114, 122, 188, 6, # left
                351, 412, 343, 277, 329, 330, 280, 352, 366,323, # right
            ]
            NOSE_INDICES = []
            
            # Get landmark coordinates for mask area
            h, w, _ = frame.shape
            mask_points = []
            for idx in MOUTH_INDICES + NOSE_INDICES:
                x = int(landmarks.landmark[idx].x * w)
                y = int(landmarks.landmark[idx].y * h)
                mask_points.append((x, y))

            left_ind = [132, 215, 172, 136, 150, 149, 176, 148, 152]  # left
            right_ind = [377, 400, 378, 379, 365, 397, 288, 361]  # right

            for idx in left_ind:
                x = int(landmarks.landmark[idx].x * w)# - 15
                y = int(landmarks.landmark[idx].y * h)# + 10
                mask_points.append((x, y))

            for idx in right_ind:
                x = int(landmarks.landmark[idx].x * w)# + 15
                y = int(landmarks.landmark[idx].y * h)# + 10
                mask_points.append((x, y))

            mask = frame
            texture = cv2.resize(texture, (mask.shape[1], mask.shape[0]))  # Resize texture to match the mask size
            # print(texture.shape)
            hull = cv2.convexHull(np.array(mask_points, np.int32))

            # Create a single-channel mask (grayscale) for the convex hull area
            hull_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.uint8)  # Single-channel mask
            cv2.fillConvexPoly(hull_mask, hull, 255)  # Fill the hull with white (255) in single-channel mask

            # Apply texture using the hull mask (only inside the convex hull)
            texture_in_hull = cv2.bitwise_and(texture, texture, mask=hull_mask)

            # Keep the original image outside the hull and blend the texture inside the hull
            mask = cv2.bitwise_and(mask, mask, mask=cv2.bitwise_not(hull_mask))  # Keep outside the hull
            mask = cv2.add(mask, texture_in_hull)  # Add texture inside the hull

            frame = mask

    # Save the resulting image with texture inside the hull
    cv2.imwrite("Masked_Image.jpg", frame)


W0000 00:00:1740171015.454991   99504 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740171015.463138   99504 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


## Half Face Mask

In [11]:
### import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Load image
# cap = cv2.imread("/projectnb/cs585bp/students/dlgirija/colorferet_extracted/00001/00001_930831_fa_a.ppm")
cap = cv2.imread('face.jpg')
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    frame = cap
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            # Define the landmark indices for the mouth and nose region
            MOUTH_INDICES = [
                227, 111, 229, 230,
                447, 340, 449, 451,
                93, 137, 123, 50, 101, 100, 47, 114, 122, 188, 6, # left
                351, 412, 343, 277, 329, 330, 280, 352, 366,323, # right
            ]
            NOSE_INDICES = []
            # Get landmark coordinates for mask area
            h, w, _ = frame.shape
            mask_points = []
            frame_points = []
            for idx in MOUTH_INDICES + NOSE_INDICES:
                x = int(landmarks.landmark[idx].x * w)
                y = int(landmarks.landmark[idx].y * h)
                mask_points.append((x, y))
                frame_points.append(idx)

            left_ind = [132,215,172,136,150,149,176,148,152] # left
            right_ind = [377,400,378,379,365,397,288,361] # right

            for idx in left_ind:
                x = int(landmarks.landmark[idx].x * w)# - 15
                y = int(landmarks.landmark[idx].y * h)# + 10
                mask_points.append((x, y))
                frame_points.append(idx)

            for idx in right_ind:
                x = int(landmarks.landmark[idx].x * w) #+ 15
                y = int(landmarks.landmark[idx].y * h) #+ 10
                mask_points.append((x, y))
                frame_points.append(idx)

            mask = frame
            hull = cv2.convexHull(np.array(mask_points, np.int32))
            cv2.fillConvexPoly(mask, hull, (100, 100, 100))
            frame = mask
            
    cv2.imwrite("Masked_Image.jpg", frame)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


W0000 00:00:1740273833.149691  243703 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740273833.160424  243710 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


## Full Face mask except EYES

In [40]:
### import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Load image
# cap = cv2.imread("/projectnb/cs585bp/students/dlgirija/colorferet_extracted/00001/00001_930831_fa_a.ppm")
cap = cv2.imread('face.jpg')
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    frame = cap
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            # Define the landmark indices for the mouth and nose region
            MOUTH_INDICES = [
                227, 111, 229, 230,
                447, 340, 449, 451,
                93, 137, 123, 50, 101, 100, 47, 114, 122, 188, 6, # left
                351, 412, 343, 277, 329, 330, 280, 352, 366,323, # right
            ]
            NOSE_INDICES = []
            # Get landmark coordinates for mask area
            h, w, _ = frame.shape
            mask_points = []
            frame_points = []
            for idx in MOUTH_INDICES + NOSE_INDICES:
                x = int(landmarks.landmark[idx].x * w)
                y = int(landmarks.landmark[idx].y * h)
                mask_points.append((x, y))
                frame_points.append(idx)

            left_ind = [132,215,172,136,150,149,176,148,152] # left
            right_ind = [377,400,378,379,365,397,288,361] # right

            for idx in left_ind:
                x = int(landmarks.landmark[idx].x * w)# - 15
                y = int(landmarks.landmark[idx].y * h)# + 10
                mask_points.append((x, y))
                frame_points.append(idx)

            for idx in right_ind:
                x = int(landmarks.landmark[idx].x * w) #+ 15
                y = int(landmarks.landmark[idx].y * h) #+ 10
                mask_points.append((x, y))
                frame_points.append(idx)

            mask = frame
            hull = cv2.convexHull(np.array(mask_points, np.int32))
            cv2.fillConvexPoly(mask, hull, (100, 100, 100))
            frame = mask

            EYE_INDICES = [
                21, 54, 103, 67, 109, 10, 297, 332, 284, 251, 389, 338,
                68, 104, 69, 151, 299, 298, 301, 337, 333, 69, 71, 162, 108
            ]
            
            mask_points = []
            frame_points = []
            for idx in EYE_INDICES:
                x = int(landmarks.landmark[idx].x * w)# - 15
                y = int(landmarks.landmark[idx].y * h)# + 10
                mask_points.append((x, y))
                frame_points.append(idx)
                
            mask = frame
            hull = cv2.convexHull(np.array(mask_points, np.int32))
            cv2.fillConvexPoly(mask, hull, (100, 100, 100))
            frame = mask
            cv2.fillPoly(mask, [np.array(mask_points, np.int32)], (100, 100, 100))  # Black-out mask area

            EYE_SIDE_INDICES = [
                162, 127, 234, 227,
                14, 156, 141, 139, 70, 71
                # 21, 54, 103, 67, 109, 10, 297, 332, 284, 251, 389, 338,
                # 68, 104, 69, 151, 299, 298, 301, 337, 333, 69, 71, 162, 108
            ]
            
            mask_points = []
            frame_points = []
            for idx in EYE_SIDE_INDICES:
                x = int(landmarks.landmark[idx].x * w)# - 15
                y = int(landmarks.landmark[idx].y * h)# + 10
                mask_points.append((x, y))
                frame_points.append(idx)
                
            mask = frame
            hull = cv2.convexHull(np.array(mask_points, np.int32))
            cv2.fillConvexPoly(mask, hull, (100, 100, 100))
            frame = mask
            cv2.fillPoly(mask, [np.array(mask_points, np.int32)], (100, 100, 100))  # Black-out mask area

            EYE_SIDE_INDICES_RIGHT = [
                300, 383, 372, 389, 298,
                356, 368, 447, 261, 449
                # 162, 127, 234, 227,
                # 14, 156, 141, 139, 70, 71
                # 21, 54, 103, 67, 109, 10, 297, 332, 284, 251, 389, 338,
                # 68, 104, 69, 151, 299, 298, 301, 337, 333, 69, 71, 162, 108
            ]
            
            mask_points = []
            frame_points = []
            for idx in EYE_SIDE_INDICES_RIGHT:
                x = int(landmarks.landmark[idx].x * w)# - 15
                y = int(landmarks.landmark[idx].y * h)# + 10
                mask_points.append((x, y))
                frame_points.append(idx)
                
            mask = frame
            hull = cv2.convexHull(np.array(mask_points, np.int32))
            cv2.fillConvexPoly(mask, hull, (100, 100, 100))
            frame = mask
            cv2.fillPoly(mask, [np.array(mask_points, np.int32)], (100, 100, 100))  # Black-out mask area
            # for i in range(len(mask_points)):
            #     cv2.putText(mask, str(frame_points[i]), mask_points[i], cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)
            #     cv2.circle(mask, mask_points[i], 2, (0, 0, 255), -1)
            # Apply mask

    
    cv2.imwrite("Masked_Image.jpg", frame)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


W0000 00:00:1740276662.216682  245869 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740276662.229512  245869 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


## Blurred Image

In [46]:
cap = cv2.imread('face.jpg')
kernel = np.ones((5,5),np.float32)/25
cap = cv2.filter2D(cap,-1,kernel)
cv2.imwrite("Masked_Image.jpg", cap)

True

## Random parts removal

In [62]:
### import cv2
import mediapipe as mp
import numpy as np
import random

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Load image
# cap = cv2.imread("/projectnb/cs585bp/students/dlgirija/colorferet_extracted/00001/00001_930831_fa_a.ppm")
cap = cv2.imread('face.jpg')
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    frame = cap
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        # for landmarks in results.multi_face_landmarks:
        inds = [i for i in range(0, 468)]
        num = random.randint(0, 467)
        INDICES = set()
        for i in range(num):
            INDICES.add(inds[random.randint(0, 467)])
        # Get landmark coordinates for mask area
        h, w, _ = frame.shape
        mask_points = []
        frame_points = []
        for idx in INDICES:
            x = int(landmarks.landmark[idx].x * w)
            y = int(landmarks.landmark[idx].y * h)
            mask_points.append((x, y))
            frame_points.append(idx)

        mask = frame
        cv2.fillPoly(mask, [np.array(mask_points, np.int32)], (100, 100, 100))  # Black-out mask area
        
        # hull = cv2.convexHull(np.array(mask_points, np.int32))
        # cv2.fillConvexPoly(mask, hull, (100, 100, 100))
        frame = mask

        print(num)
        print(INDICES)
        print(mask_points)
        
            # for i in range(len(mask_points)):
            #     cv2.putText(mask, str(frame_points[i]), mask_points[i], cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)
            #     cv2.circle(mask, mask_points[i], 2, (0, 0, 255), -1)
            # Apply mask

    
    cv2.imwrite("Masked_Image.jpg", frame)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


353
{1, 2, 5, 6, 7, 8, 10, 11, 15, 16, 17, 18, 20, 21, 24, 25, 26, 29, 34, 35, 40, 41, 42, 48, 52, 53, 56, 57, 59, 60, 61, 63, 64, 65, 67, 71, 72, 76, 77, 81, 83, 86, 87, 89, 90, 92, 97, 99, 101, 103, 104, 106, 110, 111, 114, 122, 124, 125, 126, 129, 132, 133, 134, 135, 139, 141, 142, 143, 144, 146, 147, 148, 149, 151, 152, 153, 155, 156, 157, 158, 162, 163, 165, 166, 167, 170, 174, 175, 176, 177, 178, 180, 184, 187, 188, 191, 192, 193, 196, 197, 203, 206, 207, 209, 210, 214, 216, 219, 220, 221, 222, 223, 224, 225, 228, 229, 231, 234, 237, 238, 239, 245, 247, 249, 251, 252, 253, 254, 256, 257, 258, 259, 260, 261, 262, 263, 264, 266, 270, 271, 272, 274, 277, 279, 280, 283, 285, 288, 290, 293, 295, 296, 297, 299, 300, 301, 303, 305, 306, 307, 309, 310, 312, 313, 314, 316, 317, 318, 321, 326, 333, 334, 335, 336, 337, 338, 339, 341, 342, 343, 344, 345, 346, 347, 351, 352, 355, 356, 360, 362, 363, 364, 365, 366, 368, 370, 371, 373, 375, 376, 377, 380, 382, 385, 386, 388, 390, 391, 392, 396,

W0000 00:00:1740280183.345630  247968 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740280183.356214  247968 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
